In this notebook we will investigate whether we could find channels selective to stimulus on right Vs left from evoked response in different frequency 

In [6]:
import mne
import numpy as np
import os
import pandas as pd
import pickle
import submitit
import matplotlib.pyplot as plt
import math 

In [ ]:


# Initialize variables
pid = 'b40d8c65-7ee8-4069-8b64-1a3f7907742f'
pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
versions = ['', '_csd', '_bipolar']
path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
epoch = mne.read_epochs(path_epochs, preload=True)
row = len(epoch.ch_names)
del epoch
# Prepare figure
fig, axes = plt.subplots(len(epoch.ch_names), len(versions), figsize=(15, 2*len(epoch.ch_names)), sharex=True)

for col, version in enumerate(versions):
    # Read epochs for each version
    path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched{version}.fif')
    epoch = mne.read_epochs(path_epochs, preload=True)
    meta = epoch.metadata.reset_index()
    skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
    left_index = [i for i in meta.index[meta['contrastLeft'] == 1].tolist() if i not in skewness_high]
    right_index = [i for i in meta.index[meta['contrastRight'] == 1].tolist() if i not in skewness_high]
    epoch_right = epoch[right_index]
    epoch_left = epoch[left_index]
    evoked_right = epoch_right.average().crop(tmin=-0.2, tmax=0.4)
    evoked_left = epoch_left.average().crop(tmin=-0.2, tmax=0.4)
    evoked_left_data = evoked_left.data
    evoked_right_data = evoked_right.data
    
    # Get the channel names
    ch_names = epoch.ch_names

    # Iterate through each channel and plot the evoked responses for right and left trials
    for i, ch_name in enumerate(ch_names):
        ax = axes[i, col]
        ax.plot(evoked_left.times, evoked_left_data[i], label='Left Trial', color='blue')
        ax.plot(evoked_right.times, evoked_right_data[i], label='Right Trial', color='red')
        ax.set_title(f'{ch_name} - {version}')
        ax.axvline(x=0, linestyle='--', color='k')  # Mark the stimulus onset
        if i == 0:
            ax.legend()
        if i == len(ch_names) - 1:
            ax.set_xlabel('Time (s)')
        if col == 0:
            ax.set_ylabel('Amplitude')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import mne
import numpy as np
import os
import pandas as pd
import pickle
import submitit
import matplotlib.pyplot as plt
import math

# Initialize variables
pid = 'b40d8c65-7ee8-4069-8b64-1a3f7907742f'
pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
epoch = mne.read_epochs(path_epochs, preload=True)
ch = epoch.ch_names[0:10]
epoch = epoch.pick_channels(ch)
row = math.ceil(len(epoch.ch_names) /5) 


# # Read metadata
meta = epoch.metadata.reset_index()
skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
left_index = [i for i in meta.index[meta['contrastLeft'] == 1].tolist() if i not in skewness_high]
right_index = [i for i in meta.index[meta['contrastRight'] == 1].tolist() if i not in skewness_high]
epoch_right = epoch[right_index]
epoch_left = epoch[left_index]
evoked_right = epoch_right.average().crop(tmin=-0.2, tmax=0.4)
evoked_left = epoch_left.average().crop(tmin=-0.2, tmax=0.4)
del epoch_left
del epoch_right
evoked_left_data = evoked_left.data
evoked_right_data = evoked_right.data


# Prepare figure
fig, axes = plt.subplots(row, 5, figsize=(20, 2 * row))
for i in range(row):
    for j in range(5):
        ch_name = epoch.ch_names[i*5+j]
        ax = axes[i, j]
        ax.plot(evoked_left.times, evoked_left_data[i*5+j], label='Left Trial', color='blue')
        ax.plot(evoked_right.times, evoked_right_data[i*5+j], label='Right Trial', color='red')
        ax.set_title(f'{ch_name}')
        ax.axvline(x=0, linestyle='--', color='k')  # Mark the stimulus onset
        if i == 0 and j == 0:
            ax.legend()
        if i == row - 1:
            ax.set_xlabel('Time (s)')
        if j == 0:
            ax.set_ylabel('Amplitude')

# Adjust layout
plt.tight_layout()
plt.title(f'pid = {pid}')
plt.savefig('test.png')


In [ ]:
import os
import math
import mne
import matplotlib.pyplot as plt

def plot_right_vs_left(pid):
    try:
        pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
        path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
        epoch = mne.read_epochs(path_epochs, preload=True)
        row = math.ceil(len(epoch.ch_names) / 5)

        # Read metadata
        meta = epoch.metadata.reset_index()
        skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
        left_index = [i for i in meta.index[meta['contrastLeft'] == 1].tolist() if i not in skewness_high]
        right_index = [i for i in meta.index[meta['contrastRight'] == 1].tolist() if i not in skewness_high]
        epoch_right = epoch[right_index]
        epoch_left = epoch[left_index]
        evoked_right = epoch_right.average().crop(tmin=-0.2, tmax=0.4)
        evoked_left = epoch_left.average().crop(tmin=-0.2, tmax=0.4)
        del epoch_left
        del epoch_right
        evoked_left_data = evoked_left.data
        evoked_right_data = evoked_right.data

        # Prepare figure
        fig, axes = plt.subplots(row, 5, figsize=(20, 2 * row))
        for i in range(row):
            for j in range(5):
                idx = i * 5 + j
                if idx < len(epoch.ch_names):
                    ch_name = epoch.ch_names[idx]
                    ax = axes[i, j]
                    
                    ax.plot(evoked_left.times, evoked_left_data[idx], label='Left Trial', color='blue')
                    ax.plot(evoked_right.times, evoked_right_data[idx], label='Right Trial', color='red')
                    ax.set_title(f'{ch_name}')
                    ax.axvline(x=0, linestyle='--', color='k')  # Mark the stimulus onset
                    if i == 0 and j == 0:
                        ax.legend()
                    if i == row - 1:
                        ax.set_xlabel('Time (s)')
                    if j == 0:
                        ax.set_ylabel('Amplitude')
                else:
                    continue
        # Adjust layout
        plt.tight_layout()
        plt.savefig(f'/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/_IBLworkflows/RightVsLeft/figures/{pid}.png')
    except Exception as e:
        print(f'Error: {e}')
        pass


In [ ]:
import pickle
import mne
import numpy as np
import os
import pandas as pd
import pickle
import submitit
import matplotlib.pyplot as plt
import math

print('#### Start submitting jobs #####')


for i, pid in enumerate(os.listdir('/mnt/data/AdaptiveControl/IBLrawdata/pid_data')):
    
    # executor is the submission interface (logs are dumped in the folder)
    executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
    executor.update_parameters(mem_gb=20, timeout_min=200, slurm_partition="CPU", cpus_per_task=1, slurm_comment='plot_evoked_rightVS_left')
    job = executor.submit(plot_right_vs_left, pid)


In [8]:
import os
import mne
import matplotlib.pyplot as plt

def plot_right_vs_left(pid):
    try:
        pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
        path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
        epoch = mne.read_epochs(path_epochs, preload=True)

        # Read metadata
        meta = epoch.metadata.reset_index()
        skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
        left_index = [i for i in meta.index[meta['contrastLeft'] == 1].tolist() if i not in skewness_high]
        right_index = [i for i in meta.index[meta['contrastRight'] == 1].tolist() if i not in skewness_high]
        epoch_right = epoch[right_index]
        epoch_left = epoch[left_index]
        evokeds = dict(
            right = list(epoch_right.iter_evoked()),
            left = list(epoch_left.iter_evoked()))

        # Prepare figure
        n_channels = len(epoch.ch_names)
        n_cols = 5
        n_rows = math.ceil(n_channels / n_cols)
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4 * n_rows), sharex=True, sharey=True)

        # Flatten the axes array for easier iteration
        axes = axes.flatten()

        # Plot for each channel
        for idx, ch_name in enumerate(epoch.ch_names):
            mne.viz.plot_compare_evokeds(evokeds, picks=ch_name, axes=axes[idx], show=False, combine=None, colors=['red', 'blue'])
            axes[idx].set_title(ch_name)

        # Remove empty subplots
        for ax in axes[n_channels:]:
            fig.delaxes(ax)

        # Adjust layout
        plt.tight_layout()
        plt.savefig(f'/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/_IBLworkflows/RightVsLeft/figures/{pid}_withERROR.png')
        plt.close(fig)
    except Exception as e:
        print(f'Error: {e}')
        pass


In [9]:
pid = '12c0f3a9-518e-4dac-9651-5d95c2b4730f'
executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
executor.update_parameters(mem_gb=20, timeout_min=200, slurm_partition="CPU", cpus_per_task=3, slurm_comment='plot_evoked_rightVS_left')
job = executor.submit(plot_right_vs_left, pid)

In [ ]:
pid = 'b40d8c65-7ee8-4069-8b64-1a3f7907742f'
try:
    pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
    path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
    epoch = mne.read_epochs(path_epochs, preload=True)
    epoch = epoch.pick_channels(epoch.ch_names[0:10])
    row = math.ceil(len(epoch.ch_names) / 5)

    # Read metadata
    meta = epoch.metadata.reset_index()
    skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
    left_index = [i for i in meta.index[meta['contrastLeft'] == 1].tolist() if i not in skewness_high]
    right_index = [i for i in meta.index[meta['contrastRight'] == 1].tolist() if i not in skewness_high]
    epoch_right = epoch[right_index]
    epoch_left = epoch[left_index]
    evokeds = dict(
        right = list(epoch_right.iter_evoked()),
        left = list(epoch_left.iter_evoked()))
    del epoch_left
    del epoch_right
    # Prepare figure
    fig, axes = plt.subplots(row, 5, figsize=(20, 2 * row))
    # axes = np.array(axes).reshape(row, 5) 
    for i in range(row):
        for j in range(5):
            idx = i * 5 + j
            if idx < len(epoch.ch_names):
                ch_name = epoch.ch_names[idx]
                ax = axes[i, j]
                mne.viz.plot_compare_evokeds(evokeds, combine = None , picks=idx, axes=ax, show=True)
                ax.set_title(f'{ch_name}')
            else:
                continue
    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/_IBLworkflows/RightVsLeft/figures/test.png')
except Exception as e:
    print(f'Error: {e}')
    pass